In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
# setting hyperparameters
BATCH_SIZE = 100
repeat_count = 1500    # epochs
PATH_DATASET="./"
PATH = "./"
FILE_TRAIN = PATH_DATASET + os.sep + "data.csv"
# FILE_TEST = PATH_DATASET + os.sep + "tf_test.csv"    
CSV_COLUMN_NAMES = []
numerical_feature_names = []
categorical_feature_names = []

In [3]:
CSV_COLUMN_NAMES = pd.read_csv(FILE_TRAIN, nrows=1).columns.tolist()
CSV_COLUMN_NAMES

['H', 'S', 'V', 'R', 'G', 'B', 'GR', 'CLASS']

In [4]:
train = pd.read_csv(FILE_TRAIN, names=CSV_COLUMN_NAMES, header=0)

In [5]:
train = pd.read_csv(FILE_TRAIN)

In [6]:
train_x, train_y = train, train.pop('CLASS')

In [7]:
training_features = ["H", "S", "V"]
train_x_final = train_x[training_features]

In [8]:
for column in train_x_final.columns:
    print (train[column].dtype)
    if(train[column].dtype == np.float64 or train[column].dtype == np.int64):
        numerical_feature_names.append(column)
    else:
        categorical_feature_names.append(column)

int64
int64
int64


In [9]:
print(numerical_feature_names)
print(categorical_feature_names)

['H', 'S', 'V']
[]


In [10]:
feature_columns = [tf.feature_column.numeric_column(k) for k in numerical_feature_names]
feature_columns

[_NumericColumn(key='H', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='S', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='V', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [11]:
# input_fn for training, convertion of dataframe to dataset
def train_input_fn(features, labels, batch_size, repeat_count):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(256).repeat(repeat_count).batch(batch_size)
    return dataset

In [12]:
# input_fn for evaluation and predicitions (labels can be null)
def eval_input_fn(features, labels, batch_size):
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset

In [14]:
def my_model_fn(features, labels, mode):
    if mode == tf.estimator.ModeKeys.PREDICT:
        tf.logging.info("my_model_fn: PREDICT, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.EVAL:
        tf.logging.info("my_model_fn: EVAL, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info("my_model_fn: TRAIN, {}".format(mode))
        
    initializer = tf.contrib.layers.xavier_initializer()
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    
    input_layer = tf.feature_column.input_layer(features, feature_columns)
    
    h1 = tf.layers.Dense(16, activation=tf.nn.relu, 
                            kernel_regularizer=regularizer,
                            kernel_initializer=initializer
                            )(input_layer)
    
    h2 = tf.layers.Dense(8, activation=tf.nn.relu, 
                            kernel_regularizer=regularizer,
                            kernel_initializer= initializer
                            )(h1)

    logits = tf.layers.Dense(6)(h2)
    
    # compute predictions
    predicted_classes = tf.argmax(input=logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_classes, name='acc_op')
    
    precision = tf.metrics.auc(labels, predictions=predicted_classes, name='precision_op')
    
    recall = tf.metrics.recall(labels, predictions=predicted_classes, name='recall_op')
    
    auc = tf.metrics.auc(labels, predictions=predicted_classes, name='auc_op')
    
    metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'auc': auc}
    
    # add metrics to tensorboard
    tf.summary.scalar('my_accuracy', accuracy[1])
    tf.summary.scalar('my_precision', precision[1])
    tf.summary.scalar('my_recall', recall[1])
    tf.summary.scalar('my_auc', auc[1])
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    # training op
    assert mode == tf.estimator.ModeKeys.TRAIN, "TRAIN is only ModeKey left"
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.05)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    

    # add gradients, weights and biases to tensorboard 
    grads = optimizer.compute_gradients(loss)
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)
    for var in tf.trainable_variables():
        tf.summary.histogram(var.name, var)
        
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


In [17]:
classifier = tf.estimator.Estimator(
    model_fn=my_model_fn,
    model_dir=PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd598389690>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './', '_save_summary_steps': 100}


In [21]:
classifier2 = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=6,
                                          model_dir=PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd502fbd150>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './', '_save_summary_steps': 100}


In [ ]:
classifier2.train(input_fn=lambda: train_input_fn(train_x, train_y, BATCH_SIZE, repeat_count))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model.ckpt-175740
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 175741 into ./model.ckpt.
INFO:tensorflow:loss = 15.107129, step = 175741
INFO:tensorflow:global_step/sec: 558.528
INFO:tensorflow:loss = 33.14435, step = 175841 (0.180 sec)
INFO:tensorflow:global_step/sec: 664.704
INFO:tensorflow:loss = 46.9614, step = 175941 (0.150 sec)
INFO:tensorflow:global_step/sec: 668.069
INFO:tensorflow:loss = 20.046846, step = 176041 (0.150 sec)
INFO:tensorflow:global_step/sec: 711.759
INFO:tensorflow:loss = 56.10369, step = 176141 (0.140 sec)
INFO:tensorflow:global_step/sec: 722.345
INFO:tensorflow:loss = 48.54409, step = 176241 (0.139 sec)
INFO:tensorflow:global_step/sec: 714.74
INFO:tensorflow:loss = 43.871143, step = 176341 

INFO:tensorflow:loss = 34.90139, step = 183541 (0.149 sec)
INFO:tensorflow:global_step/sec: 788.974
INFO:tensorflow:loss = 53.00529, step = 183641 (0.127 sec)
INFO:tensorflow:global_step/sec: 712.251
INFO:tensorflow:loss = 23.020304, step = 183741 (0.141 sec)
INFO:tensorflow:global_step/sec: 642.649
INFO:tensorflow:loss = 17.149204, step = 183841 (0.155 sec)
INFO:tensorflow:global_step/sec: 795.519
INFO:tensorflow:loss = 57.038876, step = 183941 (0.126 sec)
INFO:tensorflow:global_step/sec: 708.601
INFO:tensorflow:loss = 19.170338, step = 184041 (0.141 sec)
INFO:tensorflow:global_step/sec: 633.923
INFO:tensorflow:loss = 24.625082, step = 184141 (0.158 sec)
INFO:tensorflow:global_step/sec: 677.589
INFO:tensorflow:loss = 38.975304, step = 184241 (0.147 sec)
INFO:tensorflow:global_step/sec: 627.664
INFO:tensorflow:loss = 36.824867, step = 184341 (0.159 sec)
INFO:tensorflow:global_step/sec: 656.602
INFO:tensorflow:loss = 42.494343, step = 184441 (0.154 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:global_step/sec: 734.992
INFO:tensorflow:loss = 32.020172, step = 191741 (0.137 sec)
INFO:tensorflow:global_step/sec: 643.033
INFO:tensorflow:loss = 31.282173, step = 191841 (0.154 sec)
INFO:tensorflow:global_step/sec: 682.436
INFO:tensorflow:loss = 46.184513, step = 191941 (0.147 sec)
INFO:tensorflow:global_step/sec: 657.332
INFO:tensorflow:loss = 31.730946, step = 192041 (0.152 sec)
INFO:tensorflow:global_step/sec: 655.338
INFO:tensorflow:loss = 30.358994, step = 192141 (0.153 sec)
INFO:tensorflow:global_step/sec: 673.786
INFO:tensorflow:loss = 19.624586, step = 192241 (0.148 sec)
INFO:tensorflow:global_step/sec: 688.744
INFO:tensorflow:loss = 21.5272, step = 192341 (0.145 sec)
INFO:tensorflow:global_step/sec: 699.061
INFO:tensorflow:loss = 53.857967, step = 192441 (0.143 sec)
INFO:tensorflow:global_step/sec: 644.703
INFO:tensorflow:loss = 31.573206, step = 192541 (0.156 sec)
INFO:tensorflow:global_step/sec: 609.689
INFO:tensorflow:loss = 63.150013, step = 192641 (0.1

INFO:tensorflow:global_step/sec: 786.781
INFO:tensorflow:loss = 38.297493, step = 199941 (0.127 sec)
INFO:tensorflow:global_step/sec: 556.644
INFO:tensorflow:loss = 44.332745, step = 200041 (0.179 sec)
INFO:tensorflow:global_step/sec: 626.783
INFO:tensorflow:loss = 42.30215, step = 200141 (0.160 sec)
INFO:tensorflow:global_step/sec: 716.933
INFO:tensorflow:loss = 53.482876, step = 200241 (0.141 sec)
INFO:tensorflow:global_step/sec: 648.752
INFO:tensorflow:loss = 38.690434, step = 200341 (0.153 sec)
INFO:tensorflow:global_step/sec: 666.548
INFO:tensorflow:loss = 34.183575, step = 200441 (0.150 sec)
INFO:tensorflow:global_step/sec: 655.888
INFO:tensorflow:loss = 29.591648, step = 200541 (0.152 sec)
INFO:tensorflow:global_step/sec: 702.859
INFO:tensorflow:loss = 29.605288, step = 200641 (0.142 sec)
INFO:tensorflow:global_step/sec: 702.761
INFO:tensorflow:loss = 50.06716, step = 200741 (0.142 sec)
INFO:tensorflow:global_step/sec: 728.719
INFO:tensorflow:loss = 22.43887, step = 200841 (0.13

INFO:tensorflow:loss = 19.56082, step = 208041 (0.139 sec)
INFO:tensorflow:global_step/sec: 709.984
INFO:tensorflow:loss = 37.603245, step = 208141 (0.141 sec)
INFO:tensorflow:global_step/sec: 674.865
INFO:tensorflow:loss = 34.994232, step = 208241 (0.149 sec)
INFO:tensorflow:global_step/sec: 634.759
INFO:tensorflow:loss = 48.43679, step = 208341 (0.158 sec)
INFO:tensorflow:global_step/sec: 713.613
INFO:tensorflow:loss = 19.399117, step = 208441 (0.139 sec)
INFO:tensorflow:global_step/sec: 887.39
INFO:tensorflow:loss = 38.69026, step = 208541 (0.113 sec)
INFO:tensorflow:global_step/sec: 932.905
INFO:tensorflow:loss = 26.178173, step = 208641 (0.107 sec)
INFO:tensorflow:global_step/sec: 876.216
INFO:tensorflow:loss = 27.52054, step = 208741 (0.114 sec)
INFO:tensorflow:global_step/sec: 698.115
INFO:tensorflow:loss = 11.748793, step = 208841 (0.143 sec)
INFO:tensorflow:global_step/sec: 653.748
INFO:tensorflow:loss = 31.766668, step = 208941 (0.153 sec)
INFO:tensorflow:global_step/sec: 632

INFO:tensorflow:global_step/sec: 647.056
INFO:tensorflow:loss = 23.238209, step = 216241 (0.154 sec)
INFO:tensorflow:global_step/sec: 613.392
INFO:tensorflow:loss = 60.259457, step = 216341 (0.164 sec)
INFO:tensorflow:global_step/sec: 602.207
INFO:tensorflow:loss = 39.104767, step = 216441 (0.166 sec)
INFO:tensorflow:global_step/sec: 688.867
INFO:tensorflow:loss = 41.82157, step = 216541 (0.145 sec)
INFO:tensorflow:global_step/sec: 677.094
INFO:tensorflow:loss = 8.761699, step = 216641 (0.147 sec)
INFO:tensorflow:global_step/sec: 632.851
INFO:tensorflow:loss = 18.560778, step = 216741 (0.158 sec)
INFO:tensorflow:global_step/sec: 733.375
INFO:tensorflow:loss = 21.475327, step = 216841 (0.137 sec)
INFO:tensorflow:global_step/sec: 750.007
INFO:tensorflow:loss = 21.969505, step = 216941 (0.133 sec)
INFO:tensorflow:global_step/sec: 633.745
INFO:tensorflow:loss = 13.675158, step = 217041 (0.158 sec)
INFO:tensorflow:global_step/sec: 577.991
INFO:tensorflow:loss = 23.166687, step = 217141 (0.1

INFO:tensorflow:global_step/sec: 679.939
INFO:tensorflow:loss = 43.03671, step = 224441 (0.148 sec)
INFO:tensorflow:global_step/sec: 615.23
INFO:tensorflow:loss = 28.85445, step = 224541 (0.163 sec)
INFO:tensorflow:global_step/sec: 629.477
INFO:tensorflow:loss = 39.639294, step = 224641 (0.159 sec)
INFO:tensorflow:global_step/sec: 665.217
INFO:tensorflow:loss = 19.83236, step = 224741 (0.150 sec)
INFO:tensorflow:global_step/sec: 675.945
INFO:tensorflow:loss = 13.075138, step = 224841 (0.147 sec)
INFO:tensorflow:global_step/sec: 730.12
INFO:tensorflow:loss = 24.561281, step = 224941 (0.137 sec)
INFO:tensorflow:global_step/sec: 684.537
INFO:tensorflow:loss = 23.464191, step = 225041 (0.146 sec)
INFO:tensorflow:global_step/sec: 735.332
INFO:tensorflow:loss = 15.842249, step = 225141 (0.136 sec)
INFO:tensorflow:global_step/sec: 645.499
INFO:tensorflow:loss = 29.948284, step = 225241 (0.155 sec)
INFO:tensorflow:global_step/sec: 649.839
INFO:tensorflow:loss = 33.432045, step = 225341 (0.153 

INFO:tensorflow:global_step/sec: 797.663
INFO:tensorflow:loss = 44.025562, step = 232641 (0.125 sec)
INFO:tensorflow:global_step/sec: 681.831
INFO:tensorflow:loss = 13.882031, step = 232741 (0.147 sec)
INFO:tensorflow:global_step/sec: 667.081
INFO:tensorflow:loss = 23.088306, step = 232841 (0.149 sec)
INFO:tensorflow:global_step/sec: 733.423
INFO:tensorflow:loss = 7.880246, step = 232941 (0.136 sec)
INFO:tensorflow:global_step/sec: 760.751
INFO:tensorflow:loss = 26.316088, step = 233041 (0.132 sec)
INFO:tensorflow:global_step/sec: 615.945
INFO:tensorflow:loss = 11.559531, step = 233141 (0.162 sec)
INFO:tensorflow:global_step/sec: 621.57
INFO:tensorflow:loss = 39.667522, step = 233241 (0.162 sec)
INFO:tensorflow:global_step/sec: 621.91
INFO:tensorflow:loss = 21.818148, step = 233341 (0.161 sec)
INFO:tensorflow:global_step/sec: 709.415
INFO:tensorflow:loss = 39.91126, step = 233441 (0.140 sec)
INFO:tensorflow:global_step/sec: 678.164
INFO:tensorflow:loss = 24.909697, step = 233541 (0.148

INFO:tensorflow:global_step/sec: 733.283
INFO:tensorflow:loss = 20.48858, step = 240841 (0.137 sec)
INFO:tensorflow:global_step/sec: 645.04
INFO:tensorflow:loss = 41.92349, step = 240941 (0.154 sec)
INFO:tensorflow:global_step/sec: 712.235
INFO:tensorflow:loss = 35.376507, step = 241041 (0.141 sec)
INFO:tensorflow:global_step/sec: 698.276
INFO:tensorflow:loss = 7.747937, step = 241141 (0.143 sec)
INFO:tensorflow:global_step/sec: 639.91
INFO:tensorflow:loss = 66.33396, step = 241241 (0.157 sec)
INFO:tensorflow:global_step/sec: 650.11
INFO:tensorflow:loss = 11.430705, step = 241341 (0.154 sec)
INFO:tensorflow:global_step/sec: 635.817
INFO:tensorflow:loss = 32.489956, step = 241441 (0.157 sec)
INFO:tensorflow:global_step/sec: 659.274
INFO:tensorflow:loss = 15.248, step = 241541 (0.151 sec)
INFO:tensorflow:global_step/sec: 706.03
INFO:tensorflow:loss = 50.173313, step = 241641 (0.142 sec)
INFO:tensorflow:global_step/sec: 616.641
INFO:tensorflow:loss = 21.67802, step = 241741 (0.162 sec)
IN

INFO:tensorflow:global_step/sec: 971.137
INFO:tensorflow:loss = 20.519032, step = 249041 (0.103 sec)
INFO:tensorflow:global_step/sec: 892.508
INFO:tensorflow:loss = 24.035109, step = 249141 (0.112 sec)
INFO:tensorflow:global_step/sec: 582.846
INFO:tensorflow:loss = 37.77493, step = 249241 (0.172 sec)
INFO:tensorflow:global_step/sec: 654.112
INFO:tensorflow:loss = 5.724839, step = 249341 (0.153 sec)
INFO:tensorflow:global_step/sec: 626.17
INFO:tensorflow:loss = 38.032608, step = 249441 (0.160 sec)
INFO:tensorflow:global_step/sec: 616.39
INFO:tensorflow:loss = 6.1691117, step = 249541 (0.163 sec)
INFO:tensorflow:global_step/sec: 675.06
INFO:tensorflow:loss = 56.860497, step = 249641 (0.148 sec)
INFO:tensorflow:global_step/sec: 657.32
INFO:tensorflow:loss = 15.479909, step = 249741 (0.153 sec)
INFO:tensorflow:global_step/sec: 637.576
INFO:tensorflow:loss = 68.37655, step = 249841 (0.156 sec)
INFO:tensorflow:global_step/sec: 665.279
INFO:tensorflow:loss = 21.376482, step = 249941 (0.151 se

In [18]:
classifier.train(input_fn=lambda: train_input_fn(train_x, train_y, BATCH_SIZE, repeat_count))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: TRAIN, train
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


InvalidArgumentError: assertion failed: [predictions must be in [0, 1]] [Condition x <= y did not hold element-wise:x (ArgMax:0) = ] [3 3 3...] [y (precision_op/Cast_1:0) = ] [1]
	 [[Node: precision_op/assert_less_equal/Assert/AssertGuard/Assert = Assert[T=[DT_STRING, DT_STRING, DT_INT64, DT_STRING, DT_INT64], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](precision_op/assert_less_equal/Assert/AssertGuard/Assert/Switch, precision_op/assert_less_equal/Assert/AssertGuard/Assert/data_0, precision_op/assert_less_equal/Assert/AssertGuard/Assert/data_1, precision_op/assert_less_equal/Assert/AssertGuard/Assert/Switch_1, precision_op/assert_less_equal/Assert/AssertGuard/Assert/data_3, precision_op/assert_less_equal/Assert/AssertGuard/Assert/Switch_2)]]

Caused by op u'precision_op/assert_less_equal/Assert/AssertGuard/Assert', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2